In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import datetime
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [2]:
def findd(df):
    index = df.find(',')
    if index == -1:
        index = 0
    else:
        index = index+2
    return index

def finddd(df):
    index1 = df.find('+')
    index2 = df.find('-')
    return max(index1, index2)

def fin(df):
    index = df.find(':')
    return index

def period(x):
  if x<11:
    return 'start'
  elif x<21:
    return 'middle'
  else:
    return 'end'

def determining_week(x):
  index = x.find('/')
  hour = int(x[:index])
  weekday = int(x[index+1:])
  if hour >= 9 and hour < 17 and weekday != 0 and weekday != 6:
    return 'weekday'
  elif hour >= 9 and hour < 17 and (weekday == 0 or weekday == 6):
    return 'weekend'
  else:
    return 'all week'

def determining_day(x):
  index = x.find('/')
  hour = int(x[:index])
  weekday = int(x[index+1:])
  if hour >= 9 and hour < 17 and weekday != 0 and weekday != 6:
    return 'working hours'
  elif hour >= 9 and hour < 17 and (weekday == 0 or weekday == 6):
    return 'weekend day'
  elif hour >= 17 and hour <= 23:
    return 'evening'
  elif hour <= 5:
    return 'night'
  else:
    return 'morning'

def feature_engineering(train_df):

  train_df['date'] = train_df['date'].apply(lambda x: x.replace("  "," ")) #some rows have extra space

  train_df['day'] = train_df['date'].apply(lambda x: x[findd(x):findd(x)+2].replace(" ", ""))
  train_df['day'] = train_df['day'].apply(lambda x: '0'+x if len(x)==1 else x) # to be used for datetime

  train_df['month'] = train_df['date'].apply(lambda x: x[findd(x)+2:findd(x)+6].replace(" ", ""))

  train_df['year'] = train_df['date'].apply(lambda x: x[findd(x)+6:findd(x)+11].replace(" ", ""))

  train_df['hour'] = train_df['date'].apply(lambda x: x[findd(x)+11:fin(x)].replace(" ", ""))

  train_df['minute'] = train_df['date'].apply(lambda x: x[fin(x)+1:fin(x)+3].replace(" ", ""))

  train_df['second'] = train_df['date'].apply(lambda x: x[fin(x)+4:fin(x)+6].replace(" ", ""))

  train_df['GMT'] = train_df['date'].apply(lambda x: x[finddd(x):].replace(" ", ""))

  return train_df

def month_cleaner (x):
    try:
        x = datetime.strptime(x,'%b').strftime('%m')
        return(x)
    except:
        return (-1)
def week_day_cleaner (x):
    try:
        x = datetime.strptime(x, '%d-%b-%Y').strftime('%w')
        return(x)
    except:
        return (0)
    
def hour_cleaner (x):
    try:
        if x < 12:
            x = 'AM'  
        else: x = 'PM'
        return(x)
    except:
        return (0)
def day_of_week_cleaner(x):
    try:return(determining_week(x))
    except:return(0)
    
def time_of_day_cleaner(x):
    try:return(determining_day(x))
    except:return(0)

def feature_engineering_cleaning(train_df):
    # Feature Engineering after cleaning:
  # Example: maybe if promotion more images will be used than characters so proportion of char in body will be smaller

  train_df['month_number'] = train_df['month'].apply(month_cleaner)
  
  train_df['weekday'] = (train_df['day'] +'-'+ train_df['month'] +'-'+ train_df['year']).apply(week_day_cleaner)
  # Weekday as a decimal number, where 0 is Sunday and 6 is Saturday.

  train_df['period_month'] = train_df['day'].astype(int).apply(lambda x: period(x))

  train_df['period_time'] = train_df['hour'].astype(int).apply(hour_cleaner)

  train_df['timezone'] = train_df['GMT'].apply(lambda x: x[:5]).apply(lambda x: '+0000' if len(x) != 5 else x).apply(lambda x: '+0000' if x == '-0000' else x)

  train_df['time'] = train_df['hour'].astype('str') +'/'+ train_df['weekday'].astype('str')

  train_df['day_of_week'] = train_df['time'].apply(day_of_week_cleaner)
  train_df['time_of_day'] = train_df['time'].apply(time_of_day_cleaner)

  train_df.drop('time',axis= 1, inplace=True)

  train_df = data_cleaner(train_df)

  train_df['chars_in_subject'] = train_df['chars_in_subject'].astype(int)
  train_df['chars_in_body'] = train_df['chars_in_body'].astype(int)
  train_df['hour'] = train_df['hour'].astype(int)
  train_df['month_number'] = train_df['month_number'].astype(int)
  train_df['weekday'] = train_df['weekday'].astype(int)


  train_df['chars_in_subject_proportion'] = train_df['chars_in_subject'] / (train_df['chars_in_subject'] + train_df['chars_in_body'])
  train_df['chars_in_body_proportion'] = train_df['chars_in_body'] / (train_df['chars_in_subject'] + train_df['chars_in_body'])

  train_df['urls_images_proportion'] = (train_df['urls'] + train_df['images'])/ (train_df['chars_in_body'] + train_df['urls'] + train_df['images'])

  train_df['urls_proportion_body'] = train_df['urls']/ (train_df['chars_in_body'] + 1)

  train_df['images_proportion_body'] = train_df['images']/ (train_df['chars_in_body'] + 1)

  train_df['chars_in_subject_proportion'] = train_df['chars_in_subject_proportion']

  train_df['chars_in_body_proportion'] = train_df['chars_in_body_proportion']

  train_df['urls_images_proportion'] = train_df['urls_images_proportion']

  train_df['urls_proportion_body'] = train_df['urls_proportion_body']

  train_df['images_proportion_body'] = train_df['images_proportion_body']

  return train_df

In [3]:
hour_cleaner(10)

'AM'

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv') 

In [5]:
def data_cleaner (df, chars_in_subject = train_df['chars_in_subject'].median() ):
    
    df['org'] = df['org'].fillna('org_undefinded')
    df['tld'] = df['tld'].fillna('tld_undefinded')
    df['mail_type'] = df['mail_type'].fillna('mail_type_undefinded').apply(lambda x:x.lower())
    df['chars_in_subject'] = df['chars_in_subject'].fillna(chars_in_subject)
    df.drop([ 'date', 'GMT', 'day', 'month', 'year', 'minute', 'second'],axis= 1, inplace=True)
    return (df)

def invert_str(x):
    return(x[::-1])

In [6]:
# One hot encoding the mail_type


def ohe_mail_type (df):
    unique_mail_types = df['mail_type'].apply(lambda x: x.split('/')).explode().unique()
    for mt in unique_mail_types:
        df[f'mail_type_{mt}'] = df['mail_type'].apply(lambda x: 1 if mt in x else 0)
    df.drop('mail_type',axis =1,inplace = True)
    return(df)


def ohe_others(df):

  unique_period_month = df['period_month'].unique()
  for mt in unique_period_month:
      df[f'period_month_{mt}'] = df['period_month'].apply(lambda x: 1 if mt in x else 0)
  df.drop('period_month',axis= 1, inplace=True)

  unique_period_time = df['period_time'].unique()
  for mt in unique_period_time:
      df[f'period_time_{mt}'] = df['period_time'].apply(lambda x: 1 if mt in x else 0)
  df.drop('period_time',axis= 1, inplace=True)

  unique_day_of_week = df['day_of_week'].unique()
  for mt in unique_day_of_week:
      df[f'day_of_week_{mt}'] = df['day_of_week'].apply(lambda x: 1 if mt in x else 0)
  df.drop('day_of_week',axis= 1, inplace=True)

  unique_time_of_day = df['time_of_day'].unique()
  for mt in unique_time_of_day:
      df[f'time_of_day_{mt}'] = df['time_of_day'].apply(lambda x: 1 if mt in x else 0)
  df.drop('time_of_day',axis= 1, inplace=True)

  return df

In [7]:
# Splitting tld
def tld_split(df):
    df['tld'].apply(lambda x:x.split('.')[0]).unique()
    df['tld_back'] = df['tld'].apply(lambda x: 'undefined_tld_back' if '.' not in x else invert_str(invert_str(x).split('.')[0].lower()))
    df['tld_front'] = df['tld'].apply(lambda x: 'undefined_tld_front' if '.' not in x else x.split('.')[0].lower())
    df.drop(['tld'], axis = 1, inplace = True)
    top_10_tld_back = pd.DataFrame(df[['tld_back']].groupby('tld_back').size()).sort_values(0,ascending=False).head(20).index.tolist()
    df['tld_back'] = df['tld_back'].apply(lambda x: x if x in top_10_tld_back else 'tld_back_Uncommon')
    for bk in df['tld_back'].unique().tolist():
        df[f'tld_back_{bk}'] = df['tld_back'].apply(lambda x: 1 if x == bk else 0 )
    df.drop(['tld_back'], axis = 1, inplace = True)
    return(df)

In [8]:
test_df['label'] = 'test_df'
joint_df = train_df.append(test_df, ignore_index=False) 

C:\Users\arind\AppData\Local\Temp\ipykernel_10544\1710371832.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  joint_df = train_df.append(test_df, ignore_index=False)


In [9]:
print(joint_df.shape)
print(train_df.shape)
print(test_df.shape)
# train_df = 

(114541, 14)
(80176, 14)
(34365, 14)


In [10]:
# train_df = feature_engineering(train_df)
# train_df = train_df[train_df['year'] != '131']
# train_df = train_df[train_df['month'] != '-MAR']
# train_df = feature_engineering_cleaning(train_df)
# train_df = ohe_mail_type(train_df)
# train_df = ohe_others(train_df)
# train_df = tld_split(train_df)

In [11]:
joint_df = feature_engineering(joint_df)
joint_df = feature_engineering_cleaning(joint_df)
joint_df = ohe_mail_type(joint_df)
joint_df = ohe_others(joint_df)
joint_df = tld_split(joint_df)

In [12]:
# print(joint_df.columns)
# joint_df.to_csv('joint_chk.csv')
joint_df['chars_in_subject_proportion'] 

0        0.000612
1        0.034876
2        0.001390
3        0.000024
4        0.020383
           ...   
34360    0.001502
34361    0.002141
34362    0.007620
34363    0.014454
34364    0.004410
Name: chars_in_subject_proportion, Length: 114541, dtype: float64

In [13]:
le_org = preprocessing.LabelEncoder()
joint_df['timezone'] = le_org.fit_transform(joint_df['timezone'])

In [14]:
# train_df['timezone'] = le_org.transform(train_df['timezone'])

In [15]:
le_tldfront = preprocessing.LabelEncoder()
joint_df['tld_front'] = le_tldfront.fit_transform(joint_df['tld_front'])

In [16]:
right_df = pd.read_csv('final_label_probab_df.csv')
joint_df = joint_df.merge(right_df, on='org', how='left', indicator=False)

In [17]:

# test_df = feature_engineering(test_df)

# # ONLY FOR TRAINING SET
# # A date was 13 instead of 2013
# test_df['year'] = test_df['year'].apply(lambda x: x[1:] if len(x)==5 else x)
# test_df['month'] = test_df['month'].apply(lambda x: 'Mar' if len(x)!=3 else x)
# test_df = feature_engineering_cleaning(test_df)

In [18]:
# test_df = ohe_mail_type(test_df)
# test_df = ohe_others(test_df)
# test_df = tld_split(test_df)

In [19]:
# le_org = preprocessing.LabelEncoder()
# test_df['timezone'] = le_org.transform(test_df['timezone'])

# le_tldfront = preprocessing.LabelEncoder()
# test_df['tld_front'] = le_tldfront.transform(test_df['tld_front'])

In [20]:
# right_df = pd.read_csv('final_label_probab_df.csv')
# test_df = test_df.merge(right_df, on='org', how='left', indicator=False)

In [21]:
le_org = preprocessing.LabelEncoder()
joint_df['org'] = le_tldfront.fit_transform(joint_df['org'])

In [22]:
df_dtypes = joint_df.dtypes.to_frame('dtype')
df_dtypes[df_dtypes['dtype'] == 'object']

,dtype
label,object


In [23]:
from sklearn.preprocessing import StandardScaler
col_names = ['ccs', 'images', 'urls', 'chars_in_subject', 'chars_in_body', 'hour', 'month_number', 'weekday', 'timezone']
features = joint_df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
joint_df[col_names] = features
# scaler = StandardScaler()
# joint_df[[]] = scaler.fit_transform(joint_df[['ccs', 'images', 'urls', 'chars_in_subject', 'chars_in_body', 'hour', 'month_number', 'weekday', 'timezone']])

In [24]:
for col in joint_df.drop('label',axis=1).columns:
    if joint_df[col].astype('int').max()>5:
        print(col,joint_df[col].astype('int').max())
# joint_df

Unnamed: 0 80175
org 1082
ccs 54
images 189
urls 183
chars_in_subject 16
chars_in_body 38
tld_front 238


In [25]:
joint_df[joint_df['label'] == 'test_df'].drop('label',axis=1).to_csv('test_final_scaled.csv',index=False)
joint_df[joint_df['label'] != 'test_df'].to_csv('train_final_scaled.csv',index=False)

In [ ]:
train_df.shape

(80176, 14)

In [58]:
train_df = pd.read_csv('train_final_scaled.csv')
test_df = pd.read_csv('test_final_scaled.csv') 

train_df.drop('Unnamed: 0',axis= 1, inplace=True)
test_df.drop('Unnamed: 0',axis= 1, inplace=True)

In [60]:
train_df.drop('label',axis=1)

,org,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,hour,...,tld_back_jp,tld_back_io,label_0_probab,label_1_probab,label_2_probab,label_3_probab,label_4_probab,label_5_probab,label_6_probab,label_7_probab
0,804,-0.146616,0,0.058459,0.727489,0,0,-0.073926,-0.062080,-0.103132,...,0,0,0.032828,0.497475,0.469697,0.000000,0.000000,0.00000,0.00000,0.0
1,317,-0.146616,0,-0.018750,-0.235281,0,0,1.645504,-0.102522,-0.103132,...,0,0,0.492857,0.450000,0.057143,0.000000,0.000000,0.00000,0.00000,0.0
2,1002,-0.146616,0,-0.011938,-0.150080,0,0,-0.488961,-0.090878,1.229108,...,0,0,0.448915,0.504142,0.046943,0.000000,0.000000,0.00000,0.00000,0.0
3,440,-0.146616,0,-0.021021,-0.294922,0,0,-1.081868,0.229309,-0.103132,...,0,0,0.026502,0.733216,0.001767,0.238516,0.000000,0.00000,0.00000,0.0
4,421,0.159858,0,-0.016480,-0.201201,0,0,-0.073926,-0.102840,-0.103132,...,0,0,0.190280,0.182867,0.000000,0.626853,0.000000,0.00000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80171,421,0.159858,0,-0.021021,-0.226761,0,0,1.171179,-0.103390,-1.102311,...,0,0,0.190280,0.182867,0.000000,0.626853,0.000000,0.00000,0.00000,0.0
80172,713,-0.146616,0,-0.021021,-0.277882,1,0,0.637562,-0.097010,-0.103132,...,0,0,0.427992,0.491742,0.004926,0.048392,0.013909,0.01275,0.00029,0.0
80173,432,-0.146616,0,0.040292,0.429286,0,0,-0.162862,-0.089128,-0.602721,...,0,0,0.756041,0.238303,0.003342,0.000000,0.002314,0.00000,0.00000,0.0
80174,620,-0.146616,0,0.051646,1.741380,1,1,3.483515,-0.048830,-0.436191,...,0,0,0.562974,0.294385,0.142640,0.000000,0.000000,0.00000,0.00000,0.0


In [62]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# # train_df = train_df.drop('label',axis = 1)
x_pca = train_df.drop('label',axis=1)
x_pca = x_pca.loc[:, x_pca.columns].values
# y = df.loc[:,['target']].values
x_pca = StandardScaler().fit_transform(x_pca)

# pd.DataFrame(data = x_pca, columns = train_df.columns).head()


pca = PCA(n_components=0.73)
principalComponents = pca.fit_transform(x_pca)

principalComponents.shape
ppca = pca.n_components_
print(ppca)
pcacol = []
for t in range(1,ppca+1):
    temp = 'principal component ' + str(t)
    pcacol.append(temp)

principalDf = pd.DataFrame(data = principalComponents
              , columns = pcacol)



from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     principalDf, train_df['label'], test_size=0.2, random_state=42)

35


In [64]:
principalDf['label'] = train_df['label']
# principalDf.to_csv('lda-train.csv')
principalDf.to_csv('PCA_correct_train.csv')

In [53]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# # train_df = train_df.drop('label',axis = 1)
x_pca = test_df.loc[:, test_df.columns].values
# y = df.loc[:,['target']].values
x_pca = StandardScaler().fit_transform(x_pca)

# pd.DataFrame(data = x_pca, columns = train_df.columns).head()


pca = PCA(n_components=0.73)
principalComponents = pca.fit_transform(x_pca)

principalComponents.shape
ppca = pca.n_components_
print(ppca)
pcacol = []
for t in range(1,ppca+1):
    temp = 'principal component ' + str(t)
    pcacol.append(temp)

principalDf = pd.DataFrame(data = principalComponents
              , columns = pcacol)



from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     principalDf, train_df['label'], test_size=0.2, random_state=42)

33


In [38]:
principalDf.to_csv('pca_')

(80176, 75)

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train_df.drop(['label'],axis = 1), train_df['label'], test_size=0.2, random_state=42)

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score

neigh = KNeighborsClassifier(n_neighbors=18)
neigh.fit(X_train, y_train)
pred = neigh.predict(X_test)
print(f'Acuracy of the KNN18:{accuracy_score(y_test, pred)}')
confusion_matrix(y_test, pred)
classification_report(y_test, pred)
#precision_score(y_test, pred)
#recall_score(y_test, pred)

Acuracy of the KNN18:0.7403966076328261


'              precision    recall  f1-score   support\n\n           0       0.69      0.72      0.70      3680\n           1       0.72      0.77      0.75      7429\n           2       0.59      0.49      0.53      1719\n           3       0.89      0.82      0.85      2109\n           4       0.25      0.06      0.10        50\n           5       0.69      0.35      0.46        26\n           6       0.58      0.35      0.44        31\n           7       1.00      0.95      0.97       992\n\n    accuracy                           0.74     16036\n   macro avg       0.68      0.56      0.60     16036\nweighted avg       0.74      0.74      0.74     16036\n'

In [48]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV

lda = LDA(n_components=7)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)


parameters = {'n_neighbors':list(range(19,25))}
neigh = KNeighborsClassifier()
clf = GridSearchCV(neigh, parameters)
clf.fit(X_train, y_train)
print(clf.best_params_)
pred = clf.predict(X_test)
print(f'Acuracy of the KNN{clf.best_params_}:{accuracy_score(y_test, pred)}')
print(classification_report(y_test, pred))

{'n_neighbors': 23}
Acuracy of the KNN{'n_neighbors': 23}:0.5611748565727114
              precision    recall  f1-score   support

           0       0.58      0.59      0.59      3680
           1       0.56      0.54      0.55      7429
           2       0.51      0.47      0.49      1719
           3       0.58      0.59      0.59      2109
           4       0.20      0.10      0.13        50
           5       0.15      0.08      0.10        26
           6       0.43      0.19      0.27        31
           7       0.54      0.71      0.62       992

    accuracy                           0.56     16036
   macro avg       0.44      0.41      0.42     16036
weighted avg       0.56      0.56      0.56     16036



In [34]:
X_test.to_csv('lda-test.csv')

In [29]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# test_df = test_df.drop('label',axis = 1)
x_pca = test_df.loc[:, test_df.columns].values
# y = df.loc[:,['target']].values
x_pca = StandardScaler().fit_transform(x_pca)
pd.DataFrame(data = x_pca, columns = test_df.columns).head()


pca = PCA(n_components=0.73)
principalComponents = pca.fit_transform(x_pca)
ppca = pca.n_components_
print(ppca)
pcacol = []
for t in range(1,ppca+1):
    temp = 'principal component ' + str(t)
    pcacol.append(temp)

principalDf = pd.DataFrame(data = principalComponents
              , columns = pcacol)




33


In [53]:
lda = LDA(n_components=7)
X_train = lda.fit_transform(train_df.drop(['label'],axis = 1), train_df['label'])
X_test = lda.transform(test_df)

neigh = KNeighborsClassifier(n_neighbors=23)
neigh.fit(X_train, train_df['label'])

KNeighborsClassifier(n_neighbors=23)

In [54]:
pred = neigh.predict(X_test)

In [56]:
pred_df = pd.DataFrame(pred, columns=['label'])
pred_df.to_csv("knn_submission.csv", index=True, index_label='Id')